
# Distributed

## Dask's schedulers

- "threaded": a scheduler backed by a thread pool
- "processes": a scheduler backed by a process pool
- "single-threaded" (aka "sync"): a synchronous scheduler, good for debugging
- distributed: a distributed scheduler for executing graphs on multiple machines, see below.

## Select a scheduler

```python
with dask.config.set(scheduler='processes'):
    # set temporarily fo this block only
    myvalue.compute()

dask.config.set(scheduler='processes')
# set until further notice
```

## Making a cluster

* Locally using `LocalCluster` class
* Kubernetes using https://github.com/dask/dask-kubernetes
* Job schedulers like PBS, SLURM, and SGE https://dask-jobqueue.readthedocs.io
* Start `dask-scheduler` and `dask-worker` explicitly

In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(diagnostics_port=8080)
cluster

In [ ]:
client = Client(cluster)
client

## Executing with the distributed client

* Once you instantiate a client, it's the default
* Use the dashboard to confirm
* The dashboad gives great insight into a what's happening

## Excursion: DataFrame storage

* Normally table-like data comes as CSV
* Decompressing text and parsing CSV files is expensive
* Alternatives:
 * HDF5 in the scientific work
 * Apache Parquet in the industry
* Blog Post: https://tech.blue-yonder.com/efficient-dataframe-storage-with-apache-parquet/

## Convert taxi dataset to Parquet

This gives us the chance to use the distributed scheduler

In [ ]:
import os
nytaxi_directory='/srv/taxi-data-csv'

In [ ]:
import glob

csv_files = glob.glob(os.path.join(nytaxi_directory, '*.csv'))

In [ ]:
from dask import delayed

@delayed
def read_taxi_df(filename): 
    # As usual, we need do to some essential data cleaning to get
    # the correct data types.
    df = pd.read_csv(
        csv_files[0],
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
        infer_datetime_format=True,
    )
    df['store_and_fwd_flag'] = (df['store_and_fwd_flag'] == 'Y')
    return df

In [ ]:
import dask

tasks = [(f, read_taxi_df(f)) for f in csv_files]
tasks

In [ ]:
@delayed
def store_parquet(filename, df):
    # This changes file extension and folder name
    f = filename.replace('csv', 'parquet')
    return df.to_parquet(f, engine='pyarrow')

tasks = [store_parquet(f, df) for f, df in tasks]

In [ ]:
future = client.compute(tasks)

In [ ]:
from distributed import progress

progress(future)

In [ ]:
cluster

## Remote files

* Files are not always local to the worker.
* In HPC systems, there is often a cluster filesystem 
* Otherwise:
 * Filesystems: http://dask.pydata.org/en/latest/remote-data-services.html
 * Simple Storage: https://github.com/mbr/simplekv / https://github.com/blue-yonder/storefact